In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import math
torch.manual_seed(8)
import time
import numpy as np
import gc
import sys
sys.setrecursionlimit(50000)
import pickle
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# from tensorboardX import SummaryWriter
torch.nn.Module.dump_patches = True
import copy
import pandas as pd
#then import my own modules
from AttentiveFP.AttentiveLayers_Sim_copy import Fingerprint, GRN, AFSE
from AttentiveFP import Fingerprint_viz, save_smiles_dicts, get_smiles_dicts, get_smiles_array, moltosvg_highlight

In [2]:
from rdkit import Chem
# from rdkit.Chem import AllChem
from rdkit.Chem import QED
from rdkit.Chem import rdMolDescriptors, MolSurf
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
%matplotlib inline
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import seaborn as sns; sns.set()
from IPython.display import SVG, display
import sascorer
from AttentiveFP.utils import EarlyStopping
from AttentiveFP.utils import Meter
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import AttentiveFP.Featurizer
import scipy

In [3]:
train_filename = "./data/benchmark/EC50_P31423_1_70_train.csv"
test_filename = "./data/benchmark/EC50_P31423_1_70_test.csv"
test_active = 70
val_rate = 0.2
random_seed = 68
file_list1 = train_filename.split('/')
file1 = file_list1[-1]
file1 = file1[:-10]
number = '_run_0'
model_file = "model_file/1_GAFSE_"+file1+number
log_dir = f'log/{"1_GAFSE_"+file1}'+number
result_dir = './result/1_GAFSE_'+file1+number
print(file1)
print(model_file)

EC50_P31423_1_70
model_file/1_GAFSE_EC50_P31423_1_70_run_0


In [4]:
# task_name = 'Malaria Bioactivity'
tasks = ['value']

# train_filename = "../data/active_inactive/median_active/EC50/Q99500.csv"
feature_filename = train_filename.replace('.csv','.pickle')
filename = train_filename.replace('.csv','')
prefix_filename = train_filename.split('/')[-1].replace('.csv','')
train_df = pd.read_csv(train_filename, header=0, names = ["smiles","value"],usecols=[0,1])
# train_df = train_df[1:]
# train_df = train_df.drop(0,axis=1,inplace=False) 
print(train_df[:5])
# print(train_df.iloc(1))
def add_canonical_smiles(train_df):
    smilesList = train_df.smiles.values
    print("number of all smiles: ",len(smilesList))
    atom_num_dist = []
    remained_smiles = []
    canonical_smiles_list = []
    for smiles in smilesList:
        try:        
            mol = Chem.MolFromSmiles(smiles)
            atom_num_dist.append(len(mol.GetAtoms()))
            remained_smiles.append(smiles)
            canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
        except:
            print(smiles)
            pass
    print("number of successfully processed smiles: ", len(remained_smiles))
    train_df = train_df[train_df["smiles"].isin(remained_smiles)]
    train_df['cano_smiles'] =canonical_smiles_list
    return train_df
# print(train_df)
train_df = add_canonical_smiles(train_df)

print(train_df.head())
# plt.figure(figsize=(5, 3))
# sns.set(font_scale=1.5)
# ax = sns.distplot(atom_num_dist, bins=28, kde=False)
# plt.tight_layout()
# # plt.savefig("atom_num_dist_"+prefix_filename+".png",dpi=200)
# plt.show()
# plt.close()


                                              smiles     value
0  C1=CC(=C(C=C1C(O)P(=O)(CCC(C(=O)O)N)O)OCC(=O)O... -3.397940
1  C1C2C=CC1C3C2C(=O)N(C3=O)C4=C(C=C(C=C4)NC(=O)C... -2.988070
2  COC1=C(C=CC(=C1)NC(=O)C2CCCCC2)NC(=O)C3=CC=CC=... -3.181844
3                 C(CP(=O)(CC(C(=O)O)Cl)O)C(C(=O)O)N -3.380211
4                         C1C(C1P(=O)(O)O)C(C(=O)O)N -3.973128
number of all smiles:  95
number of successfully processed smiles:  95
                                              smiles     value  \
0  C1=CC(=C(C=C1C(O)P(=O)(CCC(C(=O)O)N)O)OCC(=O)O... -3.397940   
1  C1C2C=CC1C3C2C(=O)N(C3=O)C4=C(C=C(C=C4)NC(=O)C... -2.988070   
2  COC1=C(C=CC(=C1)NC(=O)C2CCCCC2)NC(=O)C3=CC=CC=... -3.181844   
3                 C(CP(=O)(CC(C(=O)O)Cl)O)C(C(=O)O)N -3.380211   
4                         C1C(C1P(=O)(O)O)C(C(=O)O)N -3.973128   

                                         cano_smiles  
0  NC(CCP(=O)(O)C(O)c1ccc(OCC(=O)O)c(OCC(=O)O)c1)...  
1  O=C(Nc1ccc(N2C(=O)C3C4C=CC(C4)C3C2=O)c(Cl

In [5]:
start_time = str(time.ctime()).replace(':','-').replace(' ','_')

p_dropout= 0.03
fingerprint_dim = 100

weight_decay = 4.3 # also known as l2_regularization_lambda
learning_rate = 4
radius = 2 # default: 2
T = 1
per_task_output_units_num = 1 # for regression model
output_units_num = len(tasks) * per_task_output_units_num

In [6]:
test_df = pd.read_csv(test_filename,header=0,names=["smiles","value"],usecols=[0,1])
test_df = add_canonical_smiles(test_df)
for l in test_df["cano_smiles"]:
    if l in train_df["cano_smiles"]:
        print("same smiles:",l)
        
print(test_df.shape)
print(test_df.head())

number of all smiles:  185
number of successfully processed smiles:  185
(185, 3)
                                              smiles     value  \
0  COC1=C(C(=CC(=C1)C(O)P(=O)(CCC(C(=O)O)N)O)Cl)O... -2.113943   
1  C1=CC=NC(=C1)C(=O)NC2=CC(=C(C=C2)NC(=O)C3=CC=C... -3.053078   
2  COC1=C(C(=CC(=C1)C(O)P(=O)(CCC(C(=O)O)N)O)I)OC... -2.176091   
3                         C1C(C1(C(=O)O)N)CP(=O)(O)O -1.477121   
4  CC1(CC(=O)N(C1=O)C2=C(C=C(C=C2)NC(=O)C3=CC=CC=... -3.365488   

                                        cano_smiles  
0  COc1cc(C(O)P(=O)(O)CCC(N)C(=O)O)cc(Cl)c1OCC(=O)O  
1         O=C(Nc1ccc(NC(=O)c2cccs2)c(Cl)c1)c1ccccn1  
2   COc1cc(C(O)P(=O)(O)CCC(N)C(=O)O)cc(I)c1OCC(=O)O  
3                          NC1(C(=O)O)CC1CP(=O)(O)O  
4     CC1(C)CC(=O)N(c2ccc(NC(=O)c3ccccn3)cc2Cl)C1=O  


In [7]:
print(feature_filename)
print(filename)
total_df = pd.concat([train_df,test_df],axis=0)
total_smilesList = total_df['smiles'].values
print(len(total_smilesList))
# if os.path.isfile(feature_filename):
#     feature_dicts = pickle.load(open(feature_filename, "rb" ))
# else:
#     feature_dicts = save_smiles_dicts(smilesList,filename)
feature_dicts = save_smiles_dicts(total_smilesList,filename)
remained_df = total_df[total_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
uncovered_df = total_df.drop(remained_df.index)

./data/benchmark/EC50_P31423_1_70_train.pickle
./data/benchmark/EC50_P31423_1_70_train
280
feature dicts file saved as ./data/benchmark/EC50_P31423_1_70_train.pickle


In [8]:
val_df = train_df.sample(frac=val_rate,random_state=random_seed)
train_df = train_df.drop(val_df.index)
train_df = train_df.reset_index(drop=True)
train_df = train_df[train_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
train_df = train_df.reset_index(drop=True)
val_df = val_df[val_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
val_df = val_df.reset_index(drop=True)
test_df = test_df[test_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
test_df = test_df.reset_index(drop=True)
print(train_df.shape,val_df.shape,test_df.shape)

(76, 3) (19, 3) (185, 3)


In [9]:
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([total_df["cano_smiles"].values[0]],feature_dicts)
num_atom_features = x_atom.shape[-1]
num_bond_features = x_bonds.shape[-1]
loss_function = nn.MSELoss()
model = Fingerprint(radius, T, num_atom_features, num_bond_features,
            fingerprint_dim, output_units_num, p_dropout)
amodel = AFSE(fingerprint_dim, output_units_num, p_dropout)
gmodel = GRN(radius, T, num_atom_features, num_bond_features,
            fingerprint_dim, p_dropout)
model.cuda()
amodel.cuda()
gmodel.cuda()

# optimizer = optim.Adam([
# {'params': model.parameters(), 'lr': 10**(-learning_rate), 'weight_decay ': 10**-weight_decay}, 
# {'params': gmodel.parameters(), 'lr': 10**(-learning_rate), 'weight_decay ': 10**-weight_decay}, 
# ])

optimizer = optim.Adam(params=model.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

optimizer_AFSE = optim.Adam(params=amodel.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

# optimizer_AFSE = optim.SGD(params=amodel.parameters(), lr = 0.01, momentum=0.9)

optimizer_GRN = optim.Adam(params=gmodel.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

# tensorboard = SummaryWriter(log_dir="runs/"+start_time+"_"+prefix_filename+"_"+str(fingerprint_dim)+"_"+str(p_dropout))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
# print(params)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data.shape)
        

In [10]:
import numpy as np
from matplotlib import pyplot as plt

def sorted_show_pik(dataset, p, k, k_predict, i, acc):
    p_value = dataset[tasks[0]].astype(float).tolist()
    x = np.arange(0,len(dataset),1)
#     print('plt',dataset.head(),p[:10],k_predict,k)
#     plt.figure()
#     fig, ax1 = plt.subplots()
#     ax1.grid(False)
#     ax2 = ax1.twinx()
#     plt.grid(False)
    plt.scatter(x,p,marker='.',s=6,color='r',label='predict')
#     plt.ylabel('predict')
    plt.scatter(x,p_value,s=6,marker=',',color='blue',label='p_value')
    plt.axvline(x=k-1,ls="-",c="black")#添加垂直直线
    k_value = np.ones(len(dataset))
# #     print(EC50[k-1])
    k_value = k_value*k_predict
    plt.plot(x,k_value,'-',color='black')
    plt.ylabel('p_value')
    plt.title("epoch: {},  top-k recall: {}".format(i,acc))
    plt.legend(loc=3,fontsize=5)
    plt.show()
    

def topk_acc2(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
    
    df3 = df2[:k]  #取按预测值排完序后的前k个
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    k_true = true_sort[tasks[0]].values[k-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('k_true: ',type(k_true),k_true)
#     print('k_true: ',k_true,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=k_true,'acc_num: ',len(df3[df3['predict'].values>=k_true]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k)
    acc = len(df3[df3[tasks[0]].values>=k_true])/k #预测值前k个中真实排在前k个的个数/k
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/k
    if k>active_num:
        min_active = true_sort[tasks[0]].values[active_num-1]
        acc = len(df3[df3[tasks[0]].values>=min_active])/k
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

def topk_recall(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
        
    df3 = df2[:k]  #取按预测值排完序后的前k个，因为后面的全是-4.1
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    min_active = true_sort[tasks[0]].values[active_num-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('min_active: ',type(min_active),min_active)
#     print('min_active: ',min_active,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=min_active,'acc_num: ',len(df3[df3['predict'].values>=min_active]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k,'active_num: ',active_num)
    acc = len(df3[df3[tasks[0]].values>-4.1])/active_num #预测值前k个中真实排在前active_num个的个数/active_num
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/active_num
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

    
def topk_acc_recall(df, predict, k, active_num, show_flag=False, i=0):
    if k>active_num:
        return topk_recall(df, predict, k, active_num, show_flag, i)
    return topk_acc2(df,predict,k, active_num,show_flag,i)

def weighted_top_index(df, predict, active_num):
    weighted_acc_list=[]
    for k in np.arange(1,len(df)+1,1):
        acc, fp = topk_acc_recall(df, predict, k, active_num)
        weight = (len(df)-k)/len(df)
#         print('weight=',weight,'acc=',acc)
        weighted_acc_list.append(acc*weight)#
    weighted_acc_list = np.array(weighted_acc_list)
#     print('weighted_acc_list=',weighted_acc_list)
    return np.sum(weighted_acc_list)/weighted_acc_list.shape[0]

def AP(df, predict, active_num):
    prec = []
    rec = []
    for k in np.arange(1,len(df)+1,1):
        prec_k, fp1 = topk_acc2(df,predict,k, active_num)
        rec_k, fp2 = topk_recall(df, predict, k, active_num)
        prec.append(prec_k)
        rec.append(rec_k)
    # 取所有不同的recall对应的点处的精度值做平均
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # 计算包络线，从后往前取最大保证precise非减
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # 找出所有检测结果中recall不同的点
    i = np.where(mrec[1:] != mrec[:-1])[0]
#     print(prec)
#     print('prec='+str(prec)+'\n\n'+'rec='+str(rec))

    # and sum (\Delta recall) * prec
    # 用recall的间隔对精度作加权平均
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [11]:
def caculate_r2(y,predict):
#     print(y)
#     print(predict)
    y = torch.FloatTensor(y).reshape(-1,1)
    predict = torch.FloatTensor(predict).reshape(-1,1)
    y_mean = torch.mean(y)
    predict_mean = torch.mean(predict)
    
    y1 = torch.pow(torch.mm((y-y_mean).t(),(predict-predict_mean)),2)
    y2 = torch.mm((y-y_mean).t(),(y-y_mean))*torch.mm((predict-predict_mean).t(),(predict-predict_mean))
#     print(y1,y2)
    return y1/y2

In [12]:
from torch.autograd import Variable
def l2_norm(input, dim):
    norm = torch.norm(input, dim=dim, keepdim=True)
    output = torch.div(input, norm+1e-6)
    return output

def normalize_perturbation(d,dim=-1):
    output = l2_norm(d, dim)
    return output

def tanh(x):
    return (torch.exp(x)-torch.exp(-x))/(torch.exp(x)+torch.exp(-x))

def sigmoid(x):
    return 1/(1+torch.exp(-x))

def perturb_feature(f, model, alpha=1, lamda=10**-learning_rate, output_lr=False, output_plr=False, y=None):
    mol_prediction = model(feature=f, d=0)
    pred = mol_prediction.detach()
#     f = torch.div(f, torch.norm(f, dim=-1, keepdim=True)+1e-9)
    eps = 1e-6 * normalize_perturbation(torch.randn(f.shape))
    eps = Variable(eps, requires_grad=True)
    # Predict on randomly perturbed image
    eps_p = model(feature=f, d=eps.cuda())
    eps_p_ = model(feature=f, d=-eps.cuda())
    p_aux = nn.Sigmoid()(eps_p/(pred+1e-6))
    p_aux_ = nn.Sigmoid()(eps_p_/(pred+1e-6))
#     loss = nn.BCELoss()(abs(p_aux),torch.ones_like(p_aux))+nn.BCELoss()(abs(p_aux_),torch.ones_like(p_aux_))
    loss = loss_function(p_aux,torch.ones_like(p_aux))+loss_function(p_aux_,torch.ones_like(p_aux_))
    loss.backward(retain_graph=True)

    # Based on perturbed image, get direction of greatest error
    eps_adv = eps.grad#/10**-learning_rate
    optimizer_AFSE.zero_grad()
    # Use that direction as adversarial perturbation
    eps_adv_normed = normalize_perturbation(eps_adv)
    d_adv = lamda * eps_adv_normed.cuda()
    if output_lr:
        f_p, max_lr = model(feature=f, d=d_adv, output_lr=output_lr)
    f_p = model(feature=f, d=d_adv)
    f_p_ = model(feature=f, d=-d_adv)
    p = nn.Sigmoid()(f_p/(pred+1e-6))
    p_ = nn.Sigmoid()(f_p_/(pred+1e-6))
    vat_loss = loss_function(p,torch.ones_like(p))+loss_function(p_,torch.ones_like(p_))
    if output_lr:
        if output_plr:
            loss = loss_function(mol_prediction,y)
            loss.backward(retain_graph=True)
            optimizer_AFSE.zero_grad()
            punish_lr = torch.norm(torch.mean(eps.grad,0))
            return eps_adv, d_adv, vat_loss, mol_prediction, max_lr, punish_lr
        return eps_adv, d_adv, vat_loss, mol_prediction, max_lr
    return eps_adv, d_adv, vat_loss, mol_prediction

def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol

def d_loss(f, pred, model, y_val):
    diff_loss = 0
    length = len(pred)
    for i in range(length):
        for j in range(length):
            if j == i:
                continue
            pred_diff = model(feature_only=True, feature1=f[i], feature2=f[j])
            true_diff = y_val[i] - y_val[j]
            diff_loss += loss_function(pred_diff, torch.Tensor([true_diff]).view(-1,1))
    diff_loss = diff_loss/(length*(length-1))
    return diff_loss

In [13]:
def CE(x,y):
    c = 0
    l = len(y)
    for i in range(l):
        if y[i]==1:
            c += 1
    w1 = (l-c)/l
    w0 = c/l
    loss = -w1*y*torch.log(x+1e-6)-w0*(1-y)*torch.log(1-x+1e-6)
    loss = loss.mean(-1)
    return loss

def weighted_CE_loss(x,y):
    weight = 1/(y.detach().float().mean(0)+1e-9)
    weighted_CE = nn.CrossEntropyLoss(weight=weight)
#     atom_weights = (atom_weights-min(atom_weights))/(max(atom_weights)-min(atom_weights))
    return weighted_CE(x, torch.argmax(y,-1))

def generate_loss_function(refer_atom_list, x_atom, refer_bond_list, bond_neighbor, validity_mask, atom_list, bond_list):
    [a,b,c] = x_atom.shape
    [d,e,f,g] = bond_neighbor.shape
    ce_loss = nn.CrossEntropyLoss()
    one_hot_loss = 0
    interger_loss = 0
    binary_loss = 0
    counter_i = 0
    counter_j = 0
    validity_mask = torch.from_numpy(validity_mask).cuda()
    for i in range(a):
        l = (x_atom[i].sum(-1)!=0).sum(-1)
        one_hot_loss += weighted_CE_loss(refer_atom_list[i,:l,:16], x_atom[i,:l,:16]) - \
                        ((validity_mask[i,:l]*torch.log(1-atom_list[i,:l,:16]+1e-6)).sum(-1)/(validity_mask[i,:l].sum(-1)+1e-9)).mean(-1).mean(-1)+\
                         weighted_CE_loss(atom_list[i,:l,16:22], x_atom[i,:l,16:22])+ \
                         weighted_CE_loss(atom_list[i,:l,24:30], x_atom[i,:l,24:30])+ \
                         weighted_CE_loss(atom_list[i,:l,31:36], x_atom[i,:l,31:36])
        interger_loss += loss_function(atom_list[i,:l,23], x_atom[i,:l,23])+ \
                        loss_function(atom_list[i,:l,24], x_atom[i,:l,24])
        binary_loss += CE(atom_list[i,:l,30], x_atom[i,:l,30])+ \
                        CE(atom_list[i,:l,36], x_atom[i,:l,36])+ \
                        CE(atom_list[i,:l,37], x_atom[i,:l,37])+ \
                        CE(atom_list[i,:l,38], x_atom[i,:l,38])
        counter_i += 1
        for j in range(l):
            n = (bond_neighbor[i,j].sum(-1)!=0).sum(-1)
            if n==0:
                continue
            one_hot_loss += weighted_CE_loss(bond_list[i,j,:n,:4], bond_neighbor[i,j,:n,:4])+ \
                             weighted_CE_loss(bond_list[i,j,:n,6:], bond_neighbor[i,j,:n,6:])
            binary_loss += CE(bond_neighbor[i,j,:n,4], bond_list[i,j,:n,4])+ \
                           CE(bond_neighbor[i,j,:n,5], bond_list[i,j,:n,5])
            counter_j += 1
    one_hot_loss = one_hot_loss/(5*counter_i+2*counter_j)
    interger_loss = interger_loss/(2*counter_i)
    binary_loss = binary_loss/(4*counter_i+2*counter_j)
    total_loss = (one_hot_loss + interger_loss + binary_loss)/3
    return total_loss, one_hot_loss, interger_loss, binary_loss


def train(model, amodel, gmodel, dataset, test_df, optimizer_list, loss_function, epoch):
    model.train()
    amodel.train()
    gmodel.train()
    optimizer, optimizer_AFSE, optimizer_GRN = optimizer_list
    np.random.seed(epoch)
    max_len = np.max([len(dataset),len(test_df)])
    valList = np.arange(0,max_len)
    #shuffle them
    np.random.shuffle(valList)
    batch_list = []
    for i in range(0, max_len, batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)
    for counter, batch in enumerate(batch_list):
        batch_df = dataset.loc[batch%len(dataset),:]
        batch_test = test_df.loc[batch%len(test_df),:]
        global_step = epoch * len(batch_list) + counter
        smiles_list = batch_df.cano_smiles.values
        smiles_list_test = batch_test.cano_smiles.values
        y_val = batch_df[tasks[0]].values.astype(float)
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        x_atom_test, x_bonds_test, x_atom_index_test, x_bond_index_test, x_mask_test, smiles_to_rdkit_list_test = get_smiles_array(smiles_list_test,feature_dicts)
        activated_features, mol_feature = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),
                                                torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_activated_features=True)
#         mol_feature = torch.div(mol_feature, torch.norm(mol_feature, dim=-1, keepdim=True)+1e-9)
#         activated_features = torch.div(activated_features, torch.norm(activated_features, dim=-1, keepdim=True)+1e-9)
        refer_atom_list, refer_bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),
                                                  torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),
                                                  mol_feature=mol_feature,activated_features=activated_features.detach())
        
        x_atom = torch.Tensor(x_atom)
        x_bonds = torch.Tensor(x_bonds)
        x_bond_index = torch.cuda.LongTensor(x_bond_index)
        
        bond_neighbor = [x_bonds[i][x_bond_index[i]] for i in range(len(batch_df))]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        
        eps_adv, d_adv, vat_loss, mol_prediction, conv_lr, punish_lr = perturb_feature(mol_feature, amodel, alpha=1, 
                                                                                       lamda=10**-learning_rate, output_lr=True, 
                                                                                       output_plr=True, y=torch.Tensor(y_val).view(-1,1)) # 10**-learning_rate     
        regression_loss = loss_function(mol_prediction, torch.Tensor(y_val).view(-1,1))
#         atom_list, bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),
#                                       torch.Tensor(x_mask),mol_feature=mol_feature+d_adv/1e-6,activated_features=activated_features.detach())
#         success_smiles_batch, modified_smiles, success_batch, total_batch, reconstruction, validity, validity_mask = modify_atoms(smiles_list, x_atom, 
#                             bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list,
#                                                      refer_atom_list, refer_bond_list,topn=1)
#         reconstruction_loss, one_hot_loss, interger_loss,binary_loss = generate_loss_function(refer_atom_list, x_atom, refer_bond_list, 
#                                                                                               bond_neighbor, validity_mask, atom_list, 
#                                                                                               bond_list)
        x_atom_test = torch.Tensor(x_atom_test)
        x_bonds_test = torch.Tensor(x_bonds_test)
        x_bond_index_test = torch.cuda.LongTensor(x_bond_index_test)
        
        bond_neighbor_test = [x_bonds_test[i][x_bond_index_test[i]] for i in range(len(batch_test))]
        bond_neighbor_test = torch.stack(bond_neighbor_test, dim=0)
        activated_features_test, mol_feature_test = model(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),
                                                          torch.cuda.LongTensor(x_atom_index_test),torch.cuda.LongTensor(x_bond_index_test),
                                                          torch.Tensor(x_mask_test),output_activated_features=True)
#         mol_feature_test = torch.div(mol_feature_test, torch.norm(mol_feature_test, dim=-1, keepdim=True)+1e-9)
#         activated_features_test = torch.div(activated_features_test, torch.norm(activated_features_test, dim=-1, keepdim=True)+1e-9)
        eps_test, d_test, test_vat_loss, mol_prediction_test = perturb_feature(mol_feature_test, amodel, 
                                                                                    alpha=1, lamda=10**-learning_rate)
#         atom_list_test, bond_list_test = gmodel(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),torch.cuda.LongTensor(x_atom_index_test),
#                                                 torch.cuda.LongTensor(x_bond_index_test),torch.Tensor(x_mask_test),
#                                                 mol_feature=mol_feature_test+d_test/1e-6,activated_features=activated_features_test.detach())
#         refer_atom_list_test, refer_bond_list_test = gmodel(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),
#                                                             torch.cuda.LongTensor(x_atom_index_test),torch.cuda.LongTensor(x_bond_index_test),torch.Tensor(x_mask_test),
#                                                             mol_feature=mol_feature_test,activated_features=activated_features_test.detach())
#         success_smiles_batch_test, modified_smiles_test, success_batch_test, total_batch_test, reconstruction_test, validity_test, validity_mask_test = modify_atoms(smiles_list_test, x_atom_test, 
#                             bond_neighbor_test, atom_list_test, bond_list_test,smiles_list_test,smiles_to_rdkit_list_test,
#                                                      refer_atom_list_test, refer_bond_list_test,topn=1)
#         test_reconstruction_loss, test_one_hot_loss, test_interger_loss,test_binary_loss = generate_loss_function(atom_list_test, x_atom_test, bond_list_test, bond_neighbor_test, validity_mask_test, atom_list_test, bond_list_test)
        
        if vat_loss>1 or test_vat_loss>1:
            vat_loss = 1*(vat_loss/(vat_loss+1e-6).item())
            test_vat_loss = 1*(test_vat_loss/(test_vat_loss+1e-6).item())
        
        max_lr = 1e-3
        conv_lr = conv_lr - conv_lr**2 + 0.06 * punish_lr
        if conv_lr < max_lr and conv_lr >= 0:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = conv_lr.detach()
                AFSE_lr = conv_lr    
        elif conv_lr < 0:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = 0
                AFSE_lr = 0
        elif conv_lr >= max_lr:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = max_lr
                AFSE_lr = max_lr
        
        logger.add_scalar('loss/regression', regression_loss, global_step)
        logger.add_scalar('loss/AFSE', vat_loss, global_step)
        logger.add_scalar('loss/AFSE_test', test_vat_loss, global_step)
#         logger.add_scalar('loss/GRN', reconstruction_loss, global_step)
#         logger.add_scalar('loss/GRN_test', test_reconstruction_loss, global_step)
#         logger.add_scalar('loss/GRN_one_hot', one_hot_loss, global_step)
#         logger.add_scalar('loss/GRN_interger', interger_loss, global_step)
#         logger.add_scalar('loss/GRN_binary', binary_loss, global_step)
        logger.add_scalar('lr/max_lr', conv_lr, global_step)
        logger.add_scalar('lr/punish_lr', punish_lr, global_step)
        logger.add_scalar('lr/AFSE_lr', AFSE_lr, global_step)
    
        optimizer.zero_grad()
        optimizer_AFSE.zero_grad()
#         optimizer_GRN.zero_grad()
        loss =  regression_loss + 0.6*(vat_loss + test_vat_loss) # + reconstruction_loss + test_reconstruction_loss
        loss.backward()
        optimizer.step()
        optimizer_AFSE.step()
#         optimizer_GRN.step()

        
def clear_atom_map(mol):
    [a.ClearProp('molAtomMapNumber') for a  in mol.GetAtoms()]
    return mol

def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol
        
def modify_atoms(smiles, x_atom, bond_neighbor, atom_list, bond_list, y_smiles, smiles_to_rdkit_list,refer_atom_list, refer_bond_list,topn=1,viz=False):
    x_atom = x_atom.cpu().detach().numpy()
    bond_neighbor = bond_neighbor.cpu().detach().numpy()
    atom_list = atom_list.cpu().detach().numpy()
    bond_list = bond_list.cpu().detach().numpy()
    refer_atom_list = refer_atom_list.cpu().detach().numpy()
    refer_bond_list = refer_bond_list.cpu().detach().numpy()
    atom_symbol_sorted = np.argsort(x_atom[:,:,:16], axis=-1)
    atom_symbol_generated_sorted = np.argsort(atom_list[:,:,:16], axis=-1)
    generate_confidence_sorted = np.sort(atom_list[:,:,:16], axis=-1)
    modified_smiles = []
    success_smiles = []
    success_reconstruction = 0
    success_validity = 0
    success = [0 for i in range(topn)]
    total = [0 for i in range(topn)]
    confidence_threshold = 0.001
    validity_mask = np.zeros_like(atom_list[:,:,:16])
    symbol_list = ['B','C','N','O','F','Si','P','S','Cl','As','Se','Br','Te','I','At','other']
    symbol_to_rdkit = [4,6,7,8,9,14,15,16,17,33,34,35,52,53,85,0]
    for i in range(len(atom_list)):
        rank = 0
        top_idx = 0
        flag = 0
        first_run_flag = True
        l = (x_atom[i].sum(-1)!=0).sum(-1)
        cano_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles[i]))
        mol = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
        counter = 0
        for j in range(l): 
            if mol.GetAtomWithIdx(int(smiles_to_rdkit_list[cano_smiles][j])).GetAtomicNum() == \
                symbol_to_rdkit[refer_atom_list[i,j,:16].argmax(-1)]:
                counter += 1
#             print(f'atom#{smiles_to_rdkit_list[cano_smiles][j]}(f):',{symbol_list[k]: np.around(refer_atom_list[i,j,k],3) for k in range(16)},
#                   f'\natom#{smiles_to_rdkit_list[cano_smiles][j]}(f+d):',{symbol_list[k]: np.around(atom_list[i,j,k],3) for k in range(16)},
#                  '\n------------------------------------------------------------------------------------------------------------')
#         print('预测为每个原子的平均概率：\n',np.around(atom_list[i,:l,:16].mean(1),2))
#         print('预测为每个原子的最大概率：\n',np.around(atom_list[i,:l,:16].max(1),2))
        if counter == l:
            success_reconstruction += 1
        while not flag==topn:
            if rank == 16:
                rank = 0
                top_idx += 1
            if top_idx == l:
#                 print('没有满足条件的分子生成。')
                flag += 1
                continue
#             if np.sum((atom_symbol_sorted[i,:l,-1]!=atom_symbol_generated_sorted[i,:l,-1-rank]).astype(int))==0:
#                 print(f'根据预测的第{rank}大概率的原子构成的分子与原分子一致，原子位重置为0，生成下一个元素……')
#                 rank += 1
#                 top_idx = 0
#                 generate_index = np.argsort((atom_list[i,:l,:16]-refer_atom_list[i,:l,:16] -\
#                                              x_atom[i,:l,:16]).max(-1))[-1-top_idx]
#             print('i:',i,'top_idx:', top_idx, 'rank:',rank)
            if rank == 0:
                generate_index = np.argsort((atom_list[i,:l,:16]-refer_atom_list[i,:l,:16] -\
                                             x_atom[i,:l,:16]).max(-1))[-1-top_idx]
            atom_symbol_generated = np.argsort(atom_list[i,generate_index,:16]-\
                                                    refer_atom_list[i,generate_index,:16] -\
                                                    x_atom[i,generate_index,:16])[-1-rank]
            if atom_symbol_generated==x_atom[i,generate_index,:16].argmax(-1):
#                 print('生成了相同元素，生成下一个元素……')
                rank += 1
                continue
            generate_rdkit_index = smiles_to_rdkit_list[cano_smiles][generate_index]
            if np.sort(atom_list[i,generate_index,:16]-\
                refer_atom_list[i,generate_index,:16] -\
                x_atom[i,generate_index,:16])[-1-rank]<confidence_threshold:
#                 print(f'原子位{generate_rdkit_index}生成{symbol_list[atom_symbol_generated]}元素的置信度小于{confidence_threshold}，寻找下一个原子位……')
                top_idx += 1
                rank = 0
                continue
#             if symbol_to_rdkit[atom_symbol_generated]==6:
#                 print('生成了不推荐的C元素')
#                 rank += 1
#                 continue
            mol.GetAtomWithIdx(int(generate_rdkit_index)).SetAtomicNum(symbol_to_rdkit[atom_symbol_generated])
            print_mol = mol
            try:
                Chem.SanitizeMol(mol)
                if first_run_flag == True:
                    success_validity += 1
                total[flag] += 1
                if Chem.MolToSmiles(clear_atom_map(print_mol))==y_smiles[i]:
                    success[flag] +=1
#                     print('Congratulations!', success, total)
                    success_smiles.append(Chem.MolToSmiles(clear_atom_map(print_mol)))
                mol_init = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
#                 print("修改前的分子：", smiles[i])
#                 display(mol_init)
                modified_smiles.append(Chem.MolToSmiles(clear_atom_map(print_mol)))
#                 print(f"将第{generate_rdkit_index}个原子修改为{symbol_list[atom_symbol_generated]}的分子：", Chem.MolToSmiles(clear_atom_map(print_mol)))
#                 display(mol_with_atom_index(mol))
                mol_y = mol_with_atom_index(Chem.MolFromSmiles(y_smiles[i]))
#                 print("高活性分子：", y_smiles[i])
#                 display(mol_y)
                rank += 1
                flag += 1
            except:
#                 print(f"第{generate_rdkit_index}个原子符号修改为{symbol_list[atom_symbol_generated]}不符合规范，生成下一个元素……")
                validity_mask[i,generate_index,atom_symbol_generated] = 1
                rank += 1
                first_run_flag = False
    return success_smiles, modified_smiles, success, total, success_reconstruction, success_validity, validity_mask

def modify_bonds(smiles, x_atom, bond_neighbor, atom_list, bond_list, y_smiles, smiles_to_rdkit_list):
    x_atom = x_atom.cpu().detach().numpy()
    bond_neighbor = bond_neighbor.cpu().detach().numpy()
    atom_list = atom_list.cpu().detach().numpy()
    bond_list = bond_list.cpu().detach().numpy()
    modified_smiles = []
    for i in range(len(bond_neighbor)):
        l = (bond_neighbor[i].sum(-1).sum(-1)!=0).sum(-1)
        bond_type_sorted = np.argsort(bond_list[i,:l,:,:4], axis=-1)
        bond_type_generated_sorted = np.argsort(bond_list[i,:l,:,:4], axis=-1)
        generate_confidence_sorted = np.sort(bond_list[i,:l,:,:4], axis=-1)
        rank = 0
        top_idx = 0
        flag = 0
        while not flag==3:
            cano_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles[i]))
            if np.sum((bond_type_sorted[i,:,-1]!=bond_type_generated_sorted[:,:,-1-rank]).astype(int))==0:
                rank += 1
                top_idx = 0
            print('i:',i,'top_idx:', top_idx, 'rank:',rank)
            bond_type = bond_type_sorted[i,:,-1]
            bond_type_generated = bond_type_generated_sorted[:,:,-1-rank]
            generate_confidence = generate_confidence_sorted[:,:,-1-rank]
#             print(np.sort(generate_confidence + \
#                                     (atom_symbol!=atom_symbol_generated).astype(int), axis=-1))
            generate_index = np.argsort(generate_confidence + 
                                (bond_type!=bond_type_generated).astype(int), axis=-1)[-1-top_idx]
            bond_type_generated_one = bond_type_generated[generate_index]
            mol = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
            if generate_index >= len(smiles_to_rdkit_list[cano_smiles]):
                top_idx += 1
                continue
            generate_rdkit_index = smiles_to_rdkit_list[cano_smiles][generate_index]
            mol.GetBondWithIdx(int(generate_rdkit_index)).SetBondType(bond_type_generated_one)
            try:
                Chem.SanitizeMol(mol)
                mol_init = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
                print("修改前的分子：")
                display(mol_init)
                modified_smiles.append(mol)
                print(f"将第{generate_rdkit_index}个键修改为{atom_symbol_generated}的分子：")
                display(mol)
                mol = mol_with_atom_index(Chem.MolFromSmiles(y_smiles[i]))
                print("高活性分子：")
                display(mol)
                rank += 1
                flag += 1
            except:
                print(f"第{generate_rdkit_index}个原子符号修改为{atom_symbol_generated}不符合规范")
                top_idx += 1
    return modified_smiles
        
def eval(model, amodel, gmodel, dataset, topn=1, output_feature=False, generate=False, modify_atom=True,return_GRN_loss=False, viz=False):
    model.eval()
    amodel.eval()
    gmodel.eval()
    predict_list = []
    test_MSE_list = []
    r2_list = []
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    feature_list = []
    d_list = []
    success = [0 for i in range(topn)]
    total = [0 for i in range(topn)]
    generated_smiles = []
    success_smiles = []
    success_reconstruction = 0
    success_validity = 0
    reconstruction_loss, one_hot_loss, interger_loss, binary_loss = [0,0,0,0]
    
# #     取dataset中排序后的第k个
#     sorted_dataset = dataset.sort_values(by=tasks[0],ascending=False)
#     k_df = sorted_dataset.iloc[[k-1]]
#     k_smiles = k_df['cano_smiles'].values
#     k_value = k_df[tasks[0]].values.astype(float)    
    
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch) 
#     print(batch_list)
    for counter, batch in enumerate(batch_list):
#         print(type(batch))
        batch_df = dataset.loc[batch,:]
        smiles_list = batch_df.cano_smiles.values
        matched_smiles_list = smiles_list
#         print(batch_df)
        y_val = batch_df[tasks[0]].values.astype(float)
#         print(type(y_val))
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(matched_smiles_list,feature_dicts)
        x_atom = torch.Tensor(x_atom)
        x_bonds = torch.Tensor(x_bonds)
        x_bond_index = torch.cuda.LongTensor(x_bond_index)
        bond_neighbor = [x_bonds[i][x_bond_index[i]] for i in range(len(batch_df))]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        
        lamda=10**-learning_rate
        activated_features, mol_feature = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_activated_features=True)
#         mol_feature = torch.div(mol_feature, torch.norm(mol_feature, dim=-1, keepdim=True)+1e-9)
#         activated_features = torch.div(activated_features, torch.norm(activated_features, dim=-1, keepdim=True)+1e-9)
        eps_adv, d_adv, vat_loss, mol_prediction = perturb_feature(mol_feature, amodel, alpha=1, lamda=lamda)
#         print(mol_feature,d_adv)
        atom_list, bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),
                                      torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),
                                      torch.Tensor(x_mask),mol_feature=mol_feature+d_adv/(1e-6),activated_features=activated_features)
        refer_atom_list, refer_bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),mol_feature=mol_feature,activated_features=activated_features)
        if generate:
            if modify_atom:
                success_smiles_batch, modified_smiles, success_batch, total_batch, reconstruction, validity, validity_mask = modify_atoms(matched_smiles_list, x_atom, 
                            bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list,
                                                     refer_atom_list, refer_bond_list,topn=topn,viz=viz)
            else:
                modified_smiles = modify_bonds(matched_smiles_list, x_atom, bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list)
            generated_smiles.extend(modified_smiles)
            success_smiles.extend(success_smiles_batch)
#             for n in range(topn):
#                 success[n] += success_batch[n]
#                 total[n] += total_batch[n]
#                 print('congratulations:',success,total)
            success_reconstruction += reconstruction
            success_validity += validity
            reconstruction_loss, one_hot_loss, interger_loss, binary_loss = generate_loss_function(refer_atom_list, x_atom, refer_bond_list, bond_neighbor, validity_mask, atom_list, bond_list)
        d = d_adv.cpu().detach().numpy().tolist()
        d_list.extend(d)
        mol_feature_output = mol_feature.cpu().detach().numpy().tolist()
        feature_list.extend(mol_feature_output)
#         MAE = F.l1_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')   
#         print(type(mol_prediction))
        
        MSE = F.mse_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')
#         r2 = caculate_r2(mol_prediction, torch.Tensor(y_val).view(-1,1))
# #         r2_list.extend(r2.cpu().detach().numpy())
#         if r2!=r2:
#             r2 = torch.tensor(0)
#         r2_list.append(r2.item())
#         predict_list.extend(mol_prediction.cpu().detach().numpy())
#         print(x_mask[:2],atoms_prediction.shape, mol_prediction,MSE)
        predict_list.extend(mol_prediction.cpu().detach().numpy())
#         test_MAE_list.extend(MAE.data.squeeze().cpu().numpy())
        test_MSE_list.extend(MSE.data.view(-1,1).cpu().numpy())
#     print(r2_list)
    if generate:
        generated_num = len(generated_smiles)
        eval_num = len(dataset)
        unique = generated_num
        novelty = generated_num
        for i in range(generated_num):
            for j in range(generated_num-i-1):
                if generated_smiles[i]==generated_smiles[i+j+1]:
                    unique -= 1
            for k in range(eval_num):
                if generated_smiles[i]==dataset['smiles'].values[k]:
                    novelty -= 1
        unique_rate = unique/(generated_num+1e-9)
        novelty_rate = novelty/(generated_num+1e-9)
#         print(f'successfully/total generated molecules =', {f'Top-{i+1}': f'{success[i]}/{total[i]}' for i in range(topn)})
        return success_reconstruction/len(dataset), success_validity/len(dataset), unique_rate, novelty_rate, success_smiles, generated_smiles, caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list
    if return_GRN_loss:
        return d_list, feature_list,caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list,reconstruction_loss, one_hot_loss, interger_loss,binary_loss
    if output_feature:
        return d_list, feature_list,caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list
    return caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list

epoch = 0
max_epoch = 1000
batch_size = 10
patience = 100
stopper = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_model.pth')
stopper_afse = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_amodel.pth')
stopper_generate = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_gmodel.pth')

In [14]:
import datetime
from tensorboardX import SummaryWriter
now = datetime.datetime.now().strftime('%b%d_%H-%M-%S')
if os.path.isdir(log_dir):
    for files in os.listdir(log_dir):
        os.remove(log_dir+"/"+files)
    os.rmdir(log_dir)
logger = SummaryWriter(log_dir)
print(log_dir)

log/1_GAFSE_EC50_P31423_1_70_run_0


In [15]:
train_f_list=[]
train_mse_list=[]
train_r2_list=[]
test_f_list=[]
test_mse_list=[]
test_r2_list=[]
val_f_list=[]
val_mse_list=[]
val_r2_list=[]
epoch_list=[]
train_predict_list=[]
test_predict_list=[]
val_predict_list=[]
train_y_list=[]
test_y_list=[]
val_y_list=[]
train_d_list=[]
test_d_list=[]
val_d_list=[]


epoch = 0
optimizer_list = [optimizer, optimizer_AFSE, optimizer_GRN]
max_epoch = 1000
while epoch < max_epoch:
    train(model, amodel, gmodel, train_df, test_df, optimizer_list, loss_function, epoch)
#     print(train_df.shape,test_df.shape)
    train_d, train_f, train_r2, train_MSE, train_predict, reconstruction_loss, one_hot_loss, interger_loss,binary_loss = eval(model, amodel, gmodel, train_df,output_feature=True,return_GRN_loss=True)
    train_predict = np.array(train_predict)
    train_WTI = weighted_top_index(train_df, train_predict, len(train_df))
    train_tau, _ = scipy.stats.kendalltau(train_predict,train_df[tasks[0]].values.astype(float).tolist())
    val_d, val_f, val_r2, val_MSE, val_predict, val_reconstruction_loss, val_one_hot_loss, val_interger_loss,val_binary_loss = eval(model, amodel, gmodel, val_df,output_feature=True,return_GRN_loss=True)
    val_predict = np.array(val_predict)
    val_WTI = weighted_top_index(val_df, val_predict, len(val_df))
    val_AP = AP(val_df, val_predict, len(val_df))
    val_tau, _ = scipy.stats.kendalltau(val_predict,val_df[tasks[0]].values.astype(float).tolist())
    
    test_r2_a, test_MSE_a, test_predict_a = eval(model, amodel, gmodel, test_df[:test_active])
    test_d, test_f, test_r2, test_MSE, test_predict = eval(model, amodel, gmodel, test_df,output_feature=True)
    test_predict = np.array(test_predict)
    test_WTI = weighted_top_index(test_df, test_predict, test_active)
#     test_AP = AP(test_df, test_predict, test_active)
    test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())
    
    epoch = epoch + 1
    global_step = epoch * int(np.max([len(train_df),len(test_df)])/batch_size)
    logger.add_scalar('val/WTI', val_WTI, global_step)
    logger.add_scalar('val/AP', val_AP, global_step)
    logger.add_scalar('val/r2', val_r2, global_step)
    logger.add_scalar('val/RMSE', val_MSE**0.5, global_step)
    logger.add_scalar('val/Tau', val_tau, global_step)
#     logger.add_scalar('test/TAP', test_AP, global_step)
    logger.add_scalar('test/r2', test_r2_a, global_step)
    logger.add_scalar('test/RMSE', test_MSE_a**0.5, global_step)
    logger.add_scalar('test/Tau', test_tau, global_step)
    logger.add_scalar('val/GRN', reconstruction_loss, global_step)
    logger.add_scalar('val/GRN_one_hot', one_hot_loss, global_step)
    logger.add_scalar('val/GRN_interger', interger_loss, global_step)
    logger.add_scalar('val/GRN_binary', binary_loss, global_step)
    
    train_mse_list.append(train_MSE**0.5)
    train_r2_list.append(train_r2)
    val_mse_list.append(val_MSE**0.5)  
    val_r2_list.append(val_r2)
    train_f_list.append(train_f)
    val_f_list.append(val_f)
    test_f_list.append(test_f)
    epoch_list.append(epoch)
    train_predict_list.append(train_predict.flatten())
    test_predict_list.append(test_predict.flatten())
    val_predict_list.append(val_predict.flatten())
    train_y_list.append(train_df[tasks[0]].values)
    val_y_list.append(val_df[tasks[0]].values)
    test_y_list.append(test_df[tasks[0]].values)
    train_d_list.append(train_d)
    val_d_list.append(val_d)
    test_d_list.append(test_d)

    stop_index = - val_MSE**0.5 + val_tau 
    early_stop = stopper.step(stop_index, model)
    early_stop = stopper_afse.step(stop_index, amodel, if_print=False)
    early_stop = stopper_generate.step(stop_index, gmodel, if_print=False)
#     print('epoch {:d}/{:d}, validation {} {:.4f}, {} {:.4f},best validation {r2} {:.4f}'.format(epoch, total_epoch, 'r2', val_r2, 'mse:',val_MSE, stopper.best_score))
    print('Epoch:',epoch, 'Step:', global_step, 'Index:%.4f'%stop_index, 'R2:%.4f'%train_r2,'%.4f'%val_r2,'%.4f'%test_r2_a, 'RMSE:%.4f'%train_MSE**0.5, '%.4f'%val_MSE**0.5, 
          '%.4f'%test_MSE_a**0.5, 'Tau:%.4f'%train_tau,'%.4f'%val_tau,'%.4f'%test_tau)#, 'Tau:%.4f'%val_tau,'%.4f'%test_tau,'GRN:%.4f'%reconstruction_loss,'%.4f'%val_reconstruction_loss
    if early_stop:
        break


Epoch: 1 Step: 18 Index:-0.7459 R2:0.1032 0.0000 0.0263 RMSE:1.0184 0.7985 0.9194 Tau:-0.2306 0.0526 -0.4627
Epoch: 2 Step: 36 Index:-0.6829 R2:0.1003 0.0000 0.0239 RMSE:1.0102 0.7590 0.9013 Tau:-0.2440 0.0760 -0.4581
Epoch: 3 Step: 54 Index:-0.6141 R2:0.0938 0.0002 0.0185 RMSE:0.9186 0.7018 0.8410 Tau:-0.2475 0.0877 -0.4458
Epoch: 4 Step: 72 Index:-0.4947 R2:0.0874 0.0006 0.0124 RMSE:0.8339 0.5941 0.7542 Tau:-0.2404 0.0994 -0.4391
EarlyStopping counter: 1 out of 100
Epoch: 5 Step: 90 Index:-0.5120 R2:0.0718 0.0017 0.0028 RMSE:0.7583 0.5295 0.6961 Tau:-0.2341 0.0175 -0.4153
Epoch: 6 Step: 108 Index:-0.3914 R2:0.0378 0.0050 0.0064 RMSE:0.6861 0.4791 0.6458 Tau:-0.1170 0.0877 -0.3295
Epoch: 7 Step: 126 Index:-0.3871 R2:0.0128 0.0154 0.1029 RMSE:0.6365 0.4631 0.6181 Tau:0.0776 0.0760 0.0420
EarlyStopping counter: 1 out of 100
Epoch: 8 Step: 144 Index:-0.4369 R2:0.1099 0.0179 0.1249 RMSE:0.5980 0.4545 0.6015 Tau:0.2313 0.0175 0.3488
EarlyStopping counter: 2 out of 100
Epoch: 9 Step: 162 In

EarlyStopping counter: 13 out of 100
Epoch: 60 Step: 1080 Index:-0.3926 R2:0.4954 0.0300 0.1259 RMSE:0.4552 0.4920 0.6063 Tau:0.5182 0.0994 0.3509
EarlyStopping counter: 14 out of 100
Epoch: 61 Step: 1098 Index:-0.4037 R2:0.5027 0.0315 0.1231 RMSE:0.4703 0.5148 0.6242 Tau:0.5147 0.1111 0.3288
EarlyStopping counter: 15 out of 100
Epoch: 62 Step: 1116 Index:-0.4012 R2:0.4988 0.0319 0.1211 RMSE:0.4514 0.5006 0.6134 Tau:0.5098 0.0994 0.3472
EarlyStopping counter: 16 out of 100
Epoch: 63 Step: 1134 Index:-0.3911 R2:0.5023 0.0304 0.1247 RMSE:0.4474 0.5022 0.6118 Tau:0.5133 0.1111 0.3524
EarlyStopping counter: 17 out of 100
Epoch: 64 Step: 1152 Index:-0.3922 R2:0.5111 0.0317 0.1221 RMSE:0.4474 0.5033 0.6151 Tau:0.5091 0.1111 0.3397
EarlyStopping counter: 18 out of 100
Epoch: 65 Step: 1170 Index:-0.4018 R2:0.5112 0.0321 0.1234 RMSE:0.4499 0.5013 0.6142 Tau:0.5168 0.0994 0.3430
EarlyStopping counter: 19 out of 100
Epoch: 66 Step: 1188 Index:-0.3821 R2:0.5179 0.0339 0.1251 RMSE:0.4432 0.5049 0.6

Epoch: 117 Step: 2106 Index:-0.2522 R2:0.6425 0.0820 0.1345 RMSE:0.4103 0.5037 0.6484 Tau:0.5972 0.2515 0.3002
EarlyStopping counter: 1 out of 100
Epoch: 118 Step: 2124 Index:-0.2980 R2:0.6354 0.0706 0.1307 RMSE:0.3901 0.5027 0.6349 Tau:0.5817 0.2047 0.3093
EarlyStopping counter: 2 out of 100
Epoch: 119 Step: 2142 Index:-0.2847 R2:0.6431 0.0646 0.1329 RMSE:0.3780 0.5011 0.6230 Tau:0.6036 0.2164 0.3238
EarlyStopping counter: 3 out of 100
Epoch: 120 Step: 2160 Index:-0.3244 R2:0.6404 0.0692 0.1341 RMSE:0.3817 0.5057 0.6305 Tau:0.5852 0.1813 0.3195
EarlyStopping counter: 4 out of 100
Epoch: 121 Step: 2178 Index:-0.2995 R2:0.6463 0.0779 0.1351 RMSE:0.3881 0.5042 0.6372 Tau:0.5972 0.2047 0.3099
EarlyStopping counter: 5 out of 100
Epoch: 122 Step: 2196 Index:-0.3044 R2:0.6496 0.0710 0.1367 RMSE:0.3780 0.4974 0.6197 Tau:0.6022 0.1930 0.3253
EarlyStopping counter: 6 out of 100
Epoch: 123 Step: 2214 Index:-0.2925 R2:0.6510 0.0864 0.1329 RMSE:0.4117 0.5089 0.6546 Tau:0.5944 0.2164 0.2959
EarlySt

EarlyStopping counter: 1 out of 100
Epoch: 173 Step: 3114 Index:-0.2934 R2:0.7451 0.1016 0.1408 RMSE:0.3345 0.4863 0.6555 Tau:0.6818 0.1930 0.3274
EarlyStopping counter: 2 out of 100
Epoch: 174 Step: 3132 Index:-0.2946 R2:0.7267 0.1079 0.1347 RMSE:0.3918 0.5227 0.6954 Tau:0.6564 0.2281 0.3052
EarlyStopping counter: 3 out of 100
Epoch: 175 Step: 3150 Index:-0.2342 R2:0.7314 0.1021 0.1491 RMSE:0.3262 0.4857 0.6311 Tau:0.6748 0.2515 0.3315
EarlyStopping counter: 4 out of 100
Epoch: 176 Step: 3168 Index:-0.2425 R2:0.7450 0.1091 0.1361 RMSE:0.3198 0.4823 0.6445 Tau:0.6797 0.2398 0.3354
EarlyStopping counter: 5 out of 100
Epoch: 177 Step: 3186 Index:-0.2559 R2:0.7542 0.1153 0.1342 RMSE:0.3264 0.4722 0.6521 Tau:0.6839 0.2164 0.3274
EarlyStopping counter: 6 out of 100
Epoch: 178 Step: 3204 Index:-0.2588 R2:0.7464 0.0982 0.1393 RMSE:0.3183 0.4986 0.6512 Tau:0.6811 0.2398 0.3382
EarlyStopping counter: 7 out of 100
Epoch: 179 Step: 3222 Index:-0.2434 R2:0.7503 0.1066 0.1415 RMSE:0.3504 0.5066 0.6

EarlyStopping counter: 26 out of 100
Epoch: 230 Step: 4140 Index:-0.2161 R2:0.8008 0.1257 0.1449 RMSE:0.2951 0.5026 0.6487 Tau:0.7368 0.2865 0.3498
EarlyStopping counter: 27 out of 100
Epoch: 231 Step: 4158 Index:-0.1979 R2:0.8087 0.1368 0.1480 RMSE:0.2929 0.4728 0.6682 Tau:0.7460 0.2749 0.3319
EarlyStopping counter: 28 out of 100
Epoch: 232 Step: 4176 Index:-0.2508 R2:0.7850 0.1235 0.1370 RMSE:0.3242 0.5022 0.6953 Tau:0.7121 0.2515 0.3226
EarlyStopping counter: 29 out of 100
Epoch: 233 Step: 4194 Index:-0.2732 R2:0.7999 0.1317 0.1301 RMSE:0.3606 0.5130 0.7269 Tau:0.7262 0.2398 0.3229
EarlyStopping counter: 30 out of 100
Epoch: 234 Step: 4212 Index:-0.2166 R2:0.8045 0.1195 0.1450 RMSE:0.2822 0.5031 0.6575 Tau:0.7481 0.2865 0.3488
Epoch: 235 Step: 4230 Index:-0.1763 R2:0.8123 0.1467 0.1407 RMSE:0.2849 0.4629 0.6637 Tau:0.7467 0.2865 0.3309
EarlyStopping counter: 1 out of 100
Epoch: 236 Step: 4248 Index:-0.2324 R2:0.8103 0.1363 0.1487 RMSE:0.3058 0.4956 0.6938 Tau:0.7481 0.2632 0.3489
Ea

EarlyStopping counter: 18 out of 100
Epoch: 287 Step: 5166 Index:-0.2078 R2:0.8433 0.1628 0.1470 RMSE:0.2508 0.4709 0.6675 Tau:0.7770 0.2632 0.3879
EarlyStopping counter: 19 out of 100
Epoch: 288 Step: 5184 Index:-0.2412 R2:0.8397 0.1566 0.1510 RMSE:0.2676 0.4810 0.6907 Tau:0.7756 0.2398 0.3675
EarlyStopping counter: 20 out of 100
Epoch: 289 Step: 5202 Index:-0.1833 R2:0.8475 0.1633 0.1572 RMSE:0.2526 0.4698 0.6518 Tau:0.7855 0.2865 0.3674
EarlyStopping counter: 21 out of 100
Epoch: 290 Step: 5220 Index:-0.1943 R2:0.8438 0.1740 0.1525 RMSE:0.2715 0.4691 0.6945 Tau:0.7827 0.2749 0.3689
EarlyStopping counter: 22 out of 100
Epoch: 291 Step: 5238 Index:-0.1819 R2:0.8499 0.1694 0.1602 RMSE:0.2446 0.4684 0.6595 Tau:0.7841 0.2865 0.3762
EarlyStopping counter: 23 out of 100
Epoch: 292 Step: 5256 Index:-0.2468 R2:0.8455 0.1737 0.1496 RMSE:0.2980 0.4865 0.7147 Tau:0.7770 0.2398 0.3561
EarlyStopping counter: 24 out of 100
Epoch: 293 Step: 5274 Index:-0.1811 R2:0.8487 0.1591 0.1474 RMSE:0.2475 0.4

EarlyStopping counter: 1 out of 100
Epoch: 344 Step: 6192 Index:-0.1888 R2:0.8762 0.1786 0.1587 RMSE:0.2215 0.4870 0.7009 Tau:0.8102 0.2982 0.4274
EarlyStopping counter: 2 out of 100
Epoch: 345 Step: 6210 Index:-0.1464 R2:0.8866 0.2009 0.1559 RMSE:0.2359 0.4680 0.7170 Tau:0.8327 0.3216 0.4069
EarlyStopping counter: 3 out of 100
Epoch: 346 Step: 6228 Index:-0.1625 R2:0.8886 0.2041 0.1558 RMSE:0.2262 0.4608 0.7121 Tau:0.8228 0.2982 0.4135
EarlyStopping counter: 4 out of 100
Epoch: 347 Step: 6246 Index:-0.1771 R2:0.8889 0.2094 0.1536 RMSE:0.2593 0.4754 0.7398 Tau:0.8355 0.2982 0.4133
EarlyStopping counter: 5 out of 100
Epoch: 348 Step: 6264 Index:-0.1382 R2:0.8820 0.2034 0.1678 RMSE:0.2182 0.4598 0.6836 Tau:0.8151 0.3216 0.4141
EarlyStopping counter: 6 out of 100
Epoch: 349 Step: 6282 Index:-0.1608 R2:0.8877 0.2021 0.1596 RMSE:0.2692 0.4825 0.7371 Tau:0.8341 0.3216 0.4067
EarlyStopping counter: 7 out of 100
Epoch: 350 Step: 6300 Index:-0.1920 R2:0.8846 0.1977 0.1541 RMSE:0.2481 0.4785 0.7

EarlyStopping counter: 8 out of 100
Epoch: 401 Step: 7218 Index:-0.0961 R2:0.9179 0.2722 0.1352 RMSE:0.2118 0.4528 0.8012 Tau:0.8539 0.3567 0.4385
EarlyStopping counter: 9 out of 100
Epoch: 402 Step: 7236 Index:-0.1380 R2:0.9188 0.2584 0.1507 RMSE:0.1988 0.4713 0.7970 Tau:0.8482 0.3333 0.4510
EarlyStopping counter: 10 out of 100
Epoch: 403 Step: 7254 Index:-0.1110 R2:0.9175 0.2800 0.1389 RMSE:0.2201 0.4560 0.7954 Tau:0.8447 0.3450 0.4321
EarlyStopping counter: 11 out of 100
Epoch: 404 Step: 7272 Index:-0.1152 R2:0.9221 0.2843 0.1501 RMSE:0.2215 0.4602 0.8040 Tau:0.8475 0.3450 0.4352
EarlyStopping counter: 12 out of 100
Epoch: 405 Step: 7290 Index:-0.1706 R2:0.9124 0.2448 0.1603 RMSE:0.2696 0.5156 0.7537 Tau:0.8320 0.3450 0.4635
EarlyStopping counter: 13 out of 100
Epoch: 406 Step: 7308 Index:-0.1123 R2:0.9189 0.2763 0.1476 RMSE:0.2064 0.4573 0.7972 Tau:0.8412 0.3450 0.4385
EarlyStopping counter: 14 out of 100
Epoch: 407 Step: 7326 Index:-0.0851 R2:0.9189 0.2920 0.1423 RMSE:0.1963 0.441

EarlyStopping counter: 3 out of 100
Epoch: 458 Step: 8244 Index:-0.0600 R2:0.9484 0.3224 0.1142 RMSE:0.1475 0.4635 0.9946 Tau:0.8771 0.4035 0.4578
EarlyStopping counter: 4 out of 100
Epoch: 459 Step: 8262 Index:-0.0682 R2:0.9488 0.3226 0.1159 RMSE:0.1567 0.4717 1.0124 Tau:0.8785 0.4035 0.4575
EarlyStopping counter: 5 out of 100
Epoch: 460 Step: 8280 Index:-0.1190 R2:0.9435 0.3065 0.1195 RMSE:0.1505 0.4991 1.0170 Tau:0.8680 0.3801 0.4663
EarlyStopping counter: 6 out of 100
Epoch: 461 Step: 8298 Index:-0.0510 R2:0.9392 0.3365 0.1137 RMSE:0.1574 0.4662 1.0145 Tau:0.8567 0.4152 0.4639
EarlyStopping counter: 7 out of 100
Epoch: 462 Step: 8316 Index:-0.1309 R2:0.9429 0.3119 0.1172 RMSE:0.2000 0.4994 1.0468 Tau:0.8785 0.3684 0.4545
EarlyStopping counter: 8 out of 100
Epoch: 463 Step: 8334 Index:-0.1866 R2:0.9353 0.2997 0.1100 RMSE:0.2097 0.5082 1.0608 Tau:0.8609 0.3216 0.4546
EarlyStopping counter: 9 out of 100
Epoch: 464 Step: 8352 Index:-0.0861 R2:0.9426 0.3179 0.1193 RMSE:0.1711 0.4896 1.0

EarlyStopping counter: 59 out of 100
Epoch: 514 Step: 9252 Index:-0.1166 R2:0.9587 0.3648 0.0693 RMSE:0.1389 0.4968 1.5906 Tau:0.8863 0.3801 0.4789
EarlyStopping counter: 60 out of 100
Epoch: 515 Step: 9270 Index:-0.1331 R2:0.9637 0.3485 0.0711 RMSE:0.1597 0.5132 1.6231 Tau:0.9032 0.3801 0.4830
EarlyStopping counter: 61 out of 100
Epoch: 516 Step: 9288 Index:-0.0802 R2:0.9600 0.3592 0.0683 RMSE:0.1262 0.4837 1.6094 Tau:0.8828 0.4035 0.4827
EarlyStopping counter: 62 out of 100
Epoch: 517 Step: 9306 Index:-0.1205 R2:0.9629 0.3418 0.0697 RMSE:0.1248 0.5006 1.5962 Tau:0.8955 0.3801 0.4816
EarlyStopping counter: 63 out of 100
Epoch: 518 Step: 9324 Index:-0.1011 R2:0.9526 0.3527 0.0658 RMSE:0.1384 0.4929 1.6432 Tau:0.8828 0.3918 0.4946
EarlyStopping counter: 64 out of 100
Epoch: 519 Step: 9342 Index:-0.0753 R2:0.9607 0.3568 0.0719 RMSE:0.1252 0.4905 1.6538 Tau:0.8856 0.4152 0.4895
EarlyStopping counter: 65 out of 100
Epoch: 520 Step: 9360 Index:-0.1002 R2:0.9648 0.3506 0.0685 RMSE:0.1217 0.4

In [16]:
stopper.load_checkpoint(model)
stopper_afse.load_checkpoint(amodel)
stopper_generate.load_checkpoint(gmodel)
    
test_r2, test_MSE, test_predict = eval(model, amodel, gmodel, test_df)
test_r2_a, test_MSE_a, test_predict_a = eval(model, amodel, gmodel, test_df[:test_active])
test_r2_ina, test_MSE_ina, test_predict_ina = eval(model, amodel, gmodel, test_df[test_active:].reset_index(drop=True))
    
test_predict = np.array(test_predict)
test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())

k_list = [int(len(test_df)*0.01),int(len(test_df)*0.05),int(len(test_df)*0.1),int(len(test_df)*0.15),int(len(test_df)*0.2),int(len(test_df)*0.25),
          int(len(test_df)*0.3),int(len(test_df)*0.4),int(len(test_df)*0.5),50,100,150,200,250,300]
topk_list =[]
false_positive_rate_list = []
for k in k_list:
    a,b = topk_acc_recall(test_df, test_predict, k, test_active, False, epoch)
    topk_list.append(a)
    false_positive_rate_list.append(b)
WTI = weighted_top_index(test_df, test_predict, test_active)
ap = AP(test_df, test_predict, test_active)


In [17]:
print(' epoch:',epoch,'r2:%.4f'%test_r2_a,'RMSE:%.4f'%test_MSE_a**0.5,'WTI:%.4f'%WTI,'AP:%.4f'%ap,'Tau:%.4f'%test_tau,'\n','\n',
      'Top-1:%.4f'%topk_list[0],'Top-1-fp:%.4f'%false_positive_rate_list[0],'\n',
      'Top-5:%.4f'%topk_list[1],'Top-5-fp:%.4f'%false_positive_rate_list[1],'\n',
      'Top-10:%.4f'%topk_list[2],'Top-10-fp:%.4f'%false_positive_rate_list[2],'\n',
      'Top-15:%.4f'%topk_list[3],'Top-15-fp:%.4f'%false_positive_rate_list[3],'\n',
      'Top-20:%.4f'%topk_list[4],'Top-20-fp:%.4f'%false_positive_rate_list[4],'\n',
      'Top-25:%.4f'%topk_list[5],'Top-25-fp:%.4f'%false_positive_rate_list[5],'\n',
      'Top-30:%.4f'%topk_list[6],'Top-30-fp:%.4f'%false_positive_rate_list[6],'\n',
      'Top-40:%.4f'%topk_list[7],'Top-40-fp:%.4f'%false_positive_rate_list[7],'\n',
      'Top-50:%.4f'%topk_list[8],'Top-50-fp:%.4f'%false_positive_rate_list[8],'\n','\n',
      'Top50:%.4f'%topk_list[9],'Top50-fp:%.4f'%false_positive_rate_list[9],'\n',
      'Top100:%.4f'%topk_list[10],'Top100-fp:%.4f'%false_positive_rate_list[10],'\n',
      'Top150:%.4f'%topk_list[11],'Top150-fp:%.4f'%false_positive_rate_list[11],'\n',
      'Top200:%.4f'%topk_list[12],'Top200-fp:%.4f'%false_positive_rate_list[12],'\n',
      'Top250:%.4f'%topk_list[13],'Top250-fp:%.4f'%false_positive_rate_list[13],'\n',
      'Top300:%.4f'%topk_list[14],'Top300-fp:%.4f'%false_positive_rate_list[14],'\n')

 epoch: 555 r2:0.1187 RMSE:0.9844 WTI:0.3184 AP:0.6479 Tau:0.4624 
 
 Top-1:0.0000 Top-1-fp:0.0000 
 Top-5:0.3333 Top-5-fp:0.1111 
 Top-10:0.4444 Top-10-fp:0.1667 
 Top-15:0.5185 Top-15-fp:0.1481 
 Top-20:0.6216 Top-20-fp:0.1622 
 Top-25:0.6087 Top-25-fp:0.1957 
 Top-30:0.6545 Top-30-fp:0.2545 
 Top-40:0.6571 Top-40-fp:0.3784 
 Top-50:0.8000 Top-50-fp:0.3913 
 
 Top50:0.6400 Top50-fp:0.2400 
 Top100:0.8429 Top100-fp:0.4100 
 Top150:0.9857 Top150-fp:0.5400 
 Top200:1.0000 Top200-fp:0.5750 
 Top250:1.0000 Top250-fp:0.4600 
 Top300:1.0000 Top300-fp:0.3833 



In [18]:
# print('target_file:',train_filename)
# print('inactive_file:',test_filename)
# np.savez(result_dir, epoch_list, train_f_list, train_d_list, 
#          train_predict_list, train_y_list, val_f_list, val_d_list, val_predict_list, val_y_list, test_f_list, 
#          test_d_list, test_predict_list, test_y_list)
# sim_space = np.load(result_dir+'.npz')
# print(sim_space['arr_10'].shape)

In [19]:
# loss = loss_function(mol_prediction,y)
#             loss.backward(retain_graph=True)
#             optimizer_AFSE.zero_grad()
#             punish_lr = torch.norm(torch.mean(eps.grad,0))

# loss =  regression_loss + vat_loss + test_vat_loss

#         init_lr = 1e-4
#         max_lr = 10**-(init_lr-1)
#         conv_lr = conv_lr - conv_lr**2 + 0.1 * punish_lr
#         if conv_lr < max_lr:
#             for param_group in optimizer_AFSE.param_groups:
#                 param_group["lr"] = conv_lr.detach()
#                 AFSE_lr = conv_lr    
#         else:
#             for param_group in optimizer_AFSE.param_groups:
#                 param_group["lr"] = max_lr
#                 AFSE_lr = max_lr